# CSII 2024 Exercise 10: $H_{\infty}$ Control
&copy; 2024 ETH Zurich, Niclas Scheuer, Roy Werder, Dejan Milojevic, Pierre Suter, Stephan König ; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

## Description
This week's Jupyter notebook will include a small recap of the $H_{\infty}$ norm and an introduciton to $H_2$ and $H_{\infty}$ control.


-------

## Installation
We use the [Python library](https://python-control.readthedocs.io/en/0.9.3.post2/) `control`, which can be installed using `pip`. If you have no experience with Python, try to do some tutorials (e.g. check [this](https://docs.python.org/3/tutorial/) one). The same goes for installing Python packages using `pip`, see this [tutorial](https://packaging.python.org/en/latest/tutorials/installing-packages/). There are plenty of other Python tutorials for beginners if you do a Google/YouTube search.

If you have done all the Jupyter Notebooks leading up to this one, you should have all the necessary libraries installed. 



In [538]:
# Install the required python library with pip
!pip install control

-------

## Python Libraries

We use the following Python libraries which need to be imported. If you have no experience with the [NumPy](https://numpy.org/) library, read the documentation and do some tutorials. It is very important for matrix operations in Python.

In [539]:
# Import the required python libraries
from typing import Optional, List, Tuple
import numpy as np
import matplotlib.pyplot as plt
import control as ct
import scipy.linalg as la
np.set_printoptions(precision=6, suppress=True, linewidth=200)

This is the funtion used last week


In [540]:
def systemInfnorm(system: ct.TransferFunction) -> float:
    """
    Returns an approximation of the infinity norm of the system

    Parameters:
    - ``system`` (ct.TransferFunction): The system to compute the infinity norm of

    Returns:
    - float: The infinity norm of the system
    """
    # Create a range of frequencies to analyze over
    omega = np.linspace(-4, 4, 1000)
    H = system(omega * 1j)

    # Consider the MIMO case
    if system.ninputs > 1 or system.noutputs > 1:
        # Calculate singular values
        singular_values = [np.linalg.svd(H[..., i])[1] for i in range(len(omega))]
    # Consider the SISO case
    else:
        singular_values = [np.absolute(H[..., i]) for i in range(len(omega))]

    # Return the highest singular value
    return np.vstack(singular_values).max()

------

## Exercise 1:

As a recap from the last two weeks we will first calculate the $H_{\infty}$ norm via the state-space representation. The solution via the transfer function as introduced last week is given to check your own implementation

In [541]:
def H_inf_norm_state_space(A: np.ndarray, B: np.ndarray, C: np.ndarray, D: np.ndarray) -> float:
    """
    Returns the infinity norm of the system defined by the state space matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``C`` (np.ndarray): The output matrix of the system
    - ``D`` (np.ndarray): The feedthrough matrix of the system

    Returns:
    - float: The infinity norm of the system
    """
    
    #TODO: Implement the H_inf norm calculation for a state space system
    
    # Solution:
    gamma, step, precision = 1, 1, 1
    
    while precision > 1e-7:
        # Calculate the Hamiltonian matrix
        H = np.block([[A, 1/gamma*B@B.T], [-1/gamma*C.T@C, -A.T]])

        # Check if the eigenvalues are on the imaginary axis, then update gamma
        if any(np.isclose(np.linalg.eigvals(H).real, 0)):
            prev_gamma = gamma
            gamma = gamma + step
        else:
            prev_gamma = gamma
            gamma = gamma - step
        
        precision = abs(gamma - prev_gamma)
        step /= 2
        if gamma == 0:
            gamma = 1e-7
    
    return gamma

In [542]:
# Define the state space matrices. Feel free to change these to test the function. It also works for MIMO systems.
A = np.array([[0, 1], [-2, -1]])
B = np.array([[0], [1]])
C = np.array([[2, 1]])
D = np.array([[0]])

# Calculate the infinity norms using the method introduced last week
tf = ct.ss2tf(A, B, C, D)
tf_norm = systemInfnorm(tf)

# Calculate the infinity norm using the state space method
ss_norm = H_inf_norm_state_space(A, B, C, D)

# Print the results
print(f"Transfer function infinity norm: {tf_norm}")
print(f"State space infinity norm: {ss_norm}")
if np.isclose(tf_norm, ss_norm, rtol=1e-2):
    print("The infinity norms match!")
else:
    print("The infinity norms do not match!")

Transfer function infinity norm: 1.7853933874980654
State space infinity norm: 1.785405457019806
The infinity norms match!


------

## Exercise 2a:
You are working as a controls engineer at Autobau AG. It is your job to design a laneholding assistant. Because the lanes are denoted by striped lines (akin to this -----), the input data is noisy. As your first assignment you are tasked with coming up with an optimal observer gain L for your system, which looks as following.

$\dot{x}(t)=Ax(t)+B_ww(t)+B_uu(t) $

$z(t)=C_zx(t)+D_{zw}w(t)+D_{zu}(t) $

$y(t)=C_yx(t)+D_{yw}w(t)+D_{yu}u(t) $

Furthermore you can assume all initial conditions to be zero and also that $B_u = I$. You find the actual matrices below, but feel free to change them and test your own matrices.

In [543]:
A=np.array([[-10, 12, -1], [-1, 1.25, -2], [1, 0, -5]])

B_w=np.array([[0, 1, -4], [-1, 0, 2], [0, -1, 1]])
B_u=np.eye(3)

C_z=np.array([[1, 0, -1], [1, 0, 8], [0, 6, 1]])
C_y=np.eye(3)

D_zw=np.zeros((3,3))
D_zu=np.array([[-1, 0, -1], [2, 1, -3], [1, 10, 3]])
D_yw=np.array([[1, 0, 3], [6, 1, -0.5], [-1, 0, 1]])
D_yu=np.zeros((3,3))



In [544]:
def solve_riccati_equation(A: np.ndarray, B_w: np.ndarray, C_y: np.ndarray, R_ww: np.ndarray, D_yw: np.ndarray) -> np.ndarray:
    """
    Returns the solution to the continuous algebraic Riccati equation

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B_w`` (np.ndarray): The disturbance input matrix of the system
    - ``C_y`` (np.ndarray): The output matrix of the system
    - ``R_ww`` (np.ndarray): The disturbance covariance matrix
    - ``D_yw`` (np.ndarray): The feedthrough matrix from the disturbance to the output
    - ``Q`` (np.ndarray): The state cost matrix

    Returns:
    - np.ndarray: The solution to the continuous algebraic Riccati equation
    """
    a = (A - B_w@D_yw.T@np.linalg.inv(R_ww)@C_y).T
    b = C_y
    q = B_w@(np.eye(A.shape[0])-D_yw.T@np.linalg.inv(R_ww)@D_yw)@B_w.T
    r = np.linalg.inv(R_ww)  
    
    # to combat floating point errors
    a = np.round(a, 10)
    b = np.round(b, 10)
    q = np.round(q, 10)
    r = np.round(r, 10)
    
    Y = la.solve_continuous_are(a, b, q, r)
    return Y
    

In [545]:
def optimal_LQR(A: np.ndarray, B_w: np.ndarray, C_y: np.ndarray, D_yw: np.ndarray) -> np.ndarray:
    """
    Returns the optimal state feedback gain for the given system and cost matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``Q`` (np.ndarray): The state cost matrix
    - ``R`` (np.ndarray): The input cost matrix

    Returns:
    - np.ndarray: The optimal state feedback gain
    """
    R_ww = D_yw @ D_yw.T
    Y = solve_riccati_equation(A, B_w, C_y, R_ww, D_yw)
    L = -(Y@C_y+B_w@D_yw.T)@np.linalg.inv(R_ww)
    
    return L

Solution for this: 
We can either leave the riccati function for the students as help or have them do it themselves

In [546]:
L_optimal = optimal_LQR(A, B_w, C_y, D_yw)
print("The optimal state feedback gain is:")
print(L_optimal)

The optimal state feedback gain is:
[[ 2.375 -1.    -3.625]
 [-0.25  -0.    -1.25 ]
 [-1.625  1.     4.375]]


## Exercise 2b: 
Now repeat the same, but find the optimal controller gain F.

In [547]:
def optimal_controller_gain (A: np.ndarray, B_u: np.ndarray, C_z: np.ndarray, D_zu: np.ndarray) -> np.ndarray:
    """
    Returns the optimal state feedback gain for the given system and cost matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``Q`` (np.ndarray): The state cost matrix
    - ``R`` (np.ndarray): The input cost matrix

    Returns:
    - np.ndarray: The optimal state feedback gain
    """
    a = A
    b = B_u
    q = C_z.T@C_z
    r = D_zu.T@D_zu
    
    # to combat floating point errors
    a = np.round(a, 10)
    b = np.round(b, 10)
    q = np.round(q, 10)
    r = np.round(r, 10)
    
    X_F = la.solve_continuous_are(a, b, q, r)
    F = -np.linalg.inv(r)@B_u.T@X_F
    
    return F

In [548]:
F_optimal = optimal_controller_gain(A, B_u, C_z, D_zu)
print("Optimal controller gain:")
print(F_optimal)

Optimal controller gain:
[[-0.574304  4.174618 -1.906537]
 [ 0.192424 -1.474417  0.629292]
 [-0.389293  2.661231 -1.417598]]


## Exercise 2c: 
Great, you managed to impress your project manager by coming up with an optimal observer for the LQG problem and controller for the LQR problem. 

Now he wants you to find an $H_2$-optimal controller as denoted by K. Use the system defined in 2a. We will also return the values of the new state-space system to be able to calculate its $H_{\infty}$ norm

In [549]:
def optimal_controller(A: np.ndarray, B_w: np.ndarray, B_u: np.ndarray, C_z: np.ndarray, C_y: np.ndarray, D_zw: np.ndarray, D_zu: np.ndarray, D_yw: np.ndarray, D_yu: np.ndarray) -> np.ndarray:
    """
    Returns the optimal state feedback gain for the given system and cost matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``Q`` (np.ndarray): The state cost matrix
    - ``R`` (np.ndarray): The input cost matrix

    Returns:
    - np.ndarray: The optimal state feedback gain
    """
    # Find the optimal observer gain
    L = optimal_LQR(A, B_w, C_y, D_yw)
    
    # Find the optimal controller gain
    F = optimal_controller_gain(A, B_u, C_z, D_zu)
    

    top_left = A+B_u@F+L@C_y
    top_right = np.block([[-L, B_u]])
    bottom_left = np.block([[F], [-C_y]])
    bottom_right = np.block([
        [np.zeros((F.shape[1], L.shape[0])), np.ones((F.shape[1], B_u.shape[1]))], 
        [np.ones((C_y.shape[0], L.shape[0])), np.zeros((C_y.shape[0], B_u.shape[1]))]])
    
    K = np.block([[top_left, top_right], [bottom_left, bottom_right]])
    
    return K, top_left, top_right, bottom_left, bottom_right

In [550]:
K, A_K, B_K, C_K, D_K = optimal_controller(A, B_w, B_u, C_z, C_y, D_zw, D_zu, D_yw, D_yu)
print("Optimal H_2 controller: \n", K)

print("In state space form:")
print("A_K:\n", A_K)
print("B_K:\n", B_K)
print("C_K:\n", C_K)
print("D_K:\n", D_K)

h_inf_norm = H_inf_norm_state_space(A_K, B_K, C_K, D_K)
print(f"The H_inf norm of the optimal controller is: {h_inf_norm}")

Optimal H_2 controller: 
 [[-8.199304 15.174618 -6.531537 -2.375     1.        3.625     1.        0.        0.      ]
 [-1.057576 -0.224417 -2.620708  0.25      0.        1.25      0.        1.        0.      ]
 [-1.014293  3.661231 -2.042598  1.625    -1.       -4.375     0.        0.        1.      ]
 [-0.574304  4.174618 -1.906537  0.        0.        0.        1.        1.        1.      ]
 [ 0.192424 -1.474417  0.629292  0.        0.        0.        1.        1.        1.      ]
 [-0.389293  2.661231 -1.417598  0.        0.        0.        1.        1.        1.      ]
 [-1.       -0.       -0.        1.        1.        1.        0.        0.        0.      ]
 [-0.       -1.       -0.        1.        1.        1.        0.        0.        0.      ]
 [-0.       -0.       -1.        1.        1.        1.        0.        0.        0.      ]]
In state space form:
A_K:
 [[-8.199304 15.174618 -6.531537]
 [-1.057576 -0.224417 -2.620708]
 [-1.014293  3.661231 -2.042598]]
B_K:
 [[-

------

## Exercise 3:

Now we will use the same system as before to find a suboptimal $H_{\infty}$ controller. For simplicity you can also assume $\gamma = 0.5$.

As a challenge you can also try to implement this system in MATLAB, since it is much more optimized for $H_{\infty}$ control.

_Hint: If you are confused, look up the german wikipedia article for H_inf control [https://de.wikipedia.org/wiki/H-unendlich-Regelung] for a recipe._


In [551]:
def find_X_inf(A: np.ndarray, B_w: np.ndarray, B_u: np.ndarray, C_z: np.ndarray, gamma: float) -> np.ndarray:
    """
    Returns the infinity norm of the system defined by the state space matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``C`` (np.ndarray): The output matrix of the system
    - ``D`` (np.ndarray): The feedthrough matrix of the system

    Returns:
    - float: The infinity norm of the system
    """
    a = A
    b = np.eye(A.shape[1])
    q = C_z.T@C_z
    r = np.linalg.inv(1/gamma**2*B_w@B_w.T-B_u@B_u.T)
    
    # to combat floating point errors
    a = np.round(a, 10)
    b = np.round(b, 10)
    q = np.round(q, 10)
    r = np.round(r, 10)
    
    X = la.solve_continuous_are(a, b, q, r)
    return X
    

In [552]:
def find_Y_inf(A: np.ndarray, B_w: np.ndarray, C_z: np.ndarray, C_y: np.ndarray, gamma: float) -> np.ndarray:
    """
    Returns the infinity norm of the system defined by the state space matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``C`` (np.ndarray): The output matrix of the system
    - ``D`` (np.ndarray): The feedthrough matrix of the system

    Returns:
    - float: The infinity norm of the system
    """
    a = A
    b = np.eye(A.shape[1])
    q = B_w.T@B_w
    r = np.linalg.inv(1/gamma**2*C_z@C_z.T-C_y@C_y.T)
    
    # to combat floating point errors
    a = np.round(a, 10)
    b = np.round(b, 10)
    q = np.round(q, 10)
    r = np.round(r, 10)
    
    Y = la.solve_continuous_are(a, b, q, r)
    return Y

In [553]:
def assemble_suboptimal_K(A: np.ndarray, B_w: np.ndarray, B_u: np.ndarray, C_z: np.ndarray, C_y: np.ndarray, D_zw: np.ndarray, D_zu: np.ndarray, D_yw: np.ndarray, D_yu: np.ndarray, gamma: float) -> np.ndarray:
    """
    Returns the optimal state feedback gain for the given system and cost matrices

    Parameters:
    - ``A`` (np.ndarray): The state matrix of the system
    - ``B`` (np.ndarray): The input matrix of the system
    - ``Q`` (np.ndarray): The state cost matrix
    - ``R`` (np.ndarray): The input cost matrix

    Returns:
    - np.ndarray: The optimal state feedback gain
    """    
    # Find the suboptimal X_inf and Y_inf
    X_inf = find_X_inf(A, B_w, B_u, C_z, gamma)
    Y_inf = find_Y_inf(A, B_w, C_z, C_y, gamma)
    
    # Find the suboptimal gains
    F_inf = -B_u.T@X_inf
    L_inf = -Y_inf@C_y.T
    
    # Define helper matrices
    Z_inf = np.linalg.inv(np.eye(A.shape[0])-1/gamma**2*X_inf@Y_inf)
    
    # Assemble the suboptimal controller
    top_left = A + 1/gamma**2 *B_w@B_w.T@X_inf + B_u@F_inf + Z_inf@C_y
    top_right = -Z_inf@L_inf
    bottom_left = F_inf
    bottom_right = np.zeros((F_inf.shape[0], L_inf.shape[1]))
    
    K = np.block([[top_left, top_right], [bottom_left, bottom_right]])
    
    return K, top_left, top_right, bottom_left, bottom_right

In [554]:
gamma = 0.5 # Initial
K_suboptimal, A_K_suboptimal, B_K_suboptimal, C_K_suboptimal, D_K_suboptimal = assemble_suboptimal_K(A, B_w, B_u, C_z, C_y, D_zw, D_zu, D_yw, D_yu, gamma)
print("Suboptimal H_2 controller: \n", K_suboptimal)

# H_inf norm of the system
H_inf_norm = H_inf_norm_state_space(A_K_suboptimal, B_K_suboptimal, C_K_suboptimal, D_K_suboptimal)
print("H_inf norm of the system: ", H_inf_norm)


Suboptimal H_2 controller: 
 [[-10.633356 -32.882602 -31.767298   0.047395   0.026742  -0.089634]
 [ -0.985693  30.226659   6.573624   0.001144   0.089999  -0.024131]
 [  2.088219  11.282183   7.992176   0.009788   0.052772  -0.100206]
 [ -0.135993  -0.00939   -0.523222   0.         0.         0.      ]
 [ -0.00939   -1.549575   0.199736   0.         0.         0.      ]
 [ -0.523222   0.199736  -3.602559   0.         0.         0.      ]]
H_inf norm of the system:  0.884942452771759
